In [1]:
!rm -rf *
!gdown --id 1mlgDfPbb7GQ32zhBEZ6IkPY1YwwlVxVW 
!unzip ml-100k.zip
!gdown --id 11xsjOjSV_oc9SXjD5CHh1bE8TwA81Tmv
!unzip data.zip


Downloading...
From: https://drive.google.com/uc?id=1mlgDfPbb7GQ32zhBEZ6IkPY1YwwlVxVW
To: /content/ml-100k.zip
100% 4.92M/4.92M [00:00<00:00, 150MB/s]
Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  infla

In [2]:
import pandas as pd
import json
import numpy as np

def json2df(path):
    data = json.load(open(path,"r"))
    df = pd.DataFrame.from_dict(data)
    df = df.rename(columns={'users': 'user_id', 'items': 'item_id'})
    df.user_id= df.user_id.astype(np.int64)
    df.item_id= df.item_id.astype(np.int64)
    df.rating = df.rating.astype(np.float32)
    return df


In [3]:
train = json2df("data/100k/train_100k.json")
test = json2df("data/100k/test_100k.json")

In [4]:
movies = pd.read_csv('ml-100k/u.item', sep='|', usecols=[0].append(range(5,24)), encoding='latin-1',  header=None) 
movies.rename( columns={0:'item_id'}, inplace=True )
movies['genre_vec'] = movies[range(5,24)].values.tolist()

cols = range(1,24)
movies.drop(movies.columns[cols],axis=1,inplace=True)

In [5]:
train = train.merge(movies, how='inner', on='item_id')
test = test.merge(movies, how='inner', on='item_id')
dataset= train.append(test, ignore_index=True)
dataset

,user_id,item_id,rating,genre_vec
0,1,1,5.0,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,1,4.0,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,6,1,4.0,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,13,1,3.0,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,16,1,5.0,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
89995,437,1121,5.0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
89996,437,1262,3.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
89997,445,1591,4.0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
89998,448,1294,1.0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
import keras
import tensorflow as tf
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate, Dropout, concatenate
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import numpy as np
n_users, n_movies = dataset.user_id.max(), dataset.item_id.max()
n_latent_factors = 12

In [7]:
import keras.backend as K

def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

def newModel():

  user_input = keras.layers.Input(shape=[1],name='User')
  user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors,name='User-Embedding')(user_input))

  movie_input = keras.layers.Input(shape=[1],name='Item')
  movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors, name='Movie-Embedding')(movie_input)
  movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)

  genre_input = keras.layers.Input(shape=[19],name='Genre')
  genre_embedding = keras.layers.Embedding(19, 19, name='Genre-Embedding')(genre_input)
  genre_vec = keras.layers.Flatten(name='FlattenGenres')(genre_embedding)

  concat = concatenate([movie_vec, genre_vec])
  dense_1 = Dense(12, name ='Fully-Connected1', activation='relu')(concat)

  concat2=  concatenate([dense_1, user_vec])
  dense_2 = Dense(16, name ='Fully-Connected2', activation='relu')(concat2)
  dense_3 = Dense(12, name ='Fully-Connected3', activation='relu')(dense_2)
  prod = Dense(1, name ='Fully-Connected4', activation='relu')(dense_3)
  model = keras.Model([user_input, movie_input, genre_input], prod)

  model.compile(optimizer= 'adam', loss= rmse, metrics=['mean_absolute_error', 'mse', rmse])
  return(model)

In [8]:
models=[]
weights = []
nfold = 4
from sklearn.model_selection import KFold
kfold = KFold(nfold, shuffle=True, random_state=1)
i=1
genre_test =  np.asarray([np.asarray(row, dtype=float) for row in test.genre_vec], dtype=float)
for train_index, test_index in kfold.split(train):
    print("Training model", i)
    ftrain = train.iloc[train_index]
    ftest = train.iloc[test_index]
    model = newModel()
    genreTrain = np.asarray([np.asarray(row, dtype=float) for row in ftrain.genre_vec], dtype=float)
    genreVal = np.asarray([np.asarray(row, dtype=float) for row in ftest.genre_vec], dtype=float)
    traindata = {
        "User": ftrain.user_id,
        "Item": ftrain.item_id,
        "Genre": genreTrain
    }
    valdata = {
        "User": ftest.user_id,
        "Item": ftest.item_id,
        "Genre": genreVal
    }
    history = model.fit(traindata, ftrain.rating, validation_data=(valdata, ftest.rating), epochs=15, verbose=1)

    score = model.evaluate(valdata, ftest.rating, batch_size=test.user_id.size, verbose=1)
    print("Train model", i,"completed. Test result: - loss:", score[0]," - mean_absolute_error: ", score[1]," - mean_square_error: ", score[2], " - rmse: ", score[3],  "\n")
    models.append(model)
    weights.append(score[3]);
    i+=1

Training model 1
Epoch 1/15
1875/1875 [==============================] - 5s 2ms/step - loss: 1.0449 - mean_absolute_error: 0.8528 - mse: 1.2411 - rmse: 1.0449 - val_loss: 0.9326 - val_mean_absolute_error: 0.7462 - val_mse: 0.8909 - val_rmse: 0.9326
Epoch 2/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.9362 - mean_absolute_error: 0.7478 - mse: 0.8922 - rmse: 0.9362 - val_loss: 0.9289 - val_mean_absolute_error: 0.7458 - val_mse: 0.8835 - val_rmse: 0.9289
Epoch 3/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.9285 - mean_absolute_error: 0.7407 - mse: 0.8779 - rmse: 0.9285 - val_loss: 0.9279 - val_mean_absolute_error: 0.7480 - val_mse: 0.8810 - val_rmse: 0.9279
Epoch 4/15
1875/1875 [==============================] - 4s 2ms/step - loss: 0.9262 - mean_absolute_error: 0.7384 - mse: 0.8729 - rmse: 0.9262 - val_loss: 0.9280 - val_mean_absolute_error: 0.7390 - val_mse: 0.8831 - val_rmse: 0.9280
Epoch 5/15
1875/1875 [==============================] -

In [9]:
print('Evaluate each model on test set')
i=1
testData =  {
  "User": test.user_id,
  "Item": test.item_id,
  "Genre": genre_test
}
for model in models:
  
    score= model.evaluate(testData, test.rating, batch_size=test.user_id.size, verbose=1)
    print("Model", i, ":", "-loss:", score[0], "mean_absolute_error:", score[1])
    i+=1

Evaluate each model on test set
1/1 [==============================] - 0s 28ms/step - loss: 0.9579 - mean_absolute_error: 0.7524 - mse: 0.9175 - rmse: 0.9579
Model 1 : -loss: 0.9578740000724792 mean_absolute_error: 0.7523866891860962
1/1 [==============================] - 0s 38ms/step - loss: 0.9557 - mean_absolute_error: 0.7553 - mse: 0.9133 - rmse: 0.9557
Model 2 : -loss: 0.9556562304496765 mean_absolute_error: 0.7553102374076843
1/1 [==============================] - 0s 28ms/step - loss: 0.9536 - mean_absolute_error: 0.7502 - mse: 0.9094 - rmse: 0.9536
Model 3 : -loss: 0.9536434412002563 mean_absolute_error: 0.7501668930053711
1/1 [==============================] - 0s 29ms/step - loss: 0.9528 - mean_absolute_error: 0.7423 - mse: 0.9078 - rmse: 0.9528
Model 4 : -loss: 0.9527952075004578 mean_absolute_error: 0.7422645688056946


In [10]:
def mse(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.square(np.subtract(actual,pred)).mean() 
def mae(actual, pred):
    actual, pred = np.array(actual), np.array(pred)
    mae = np.mean(np.abs((actual - pred)))
    return mae

In [11]:
i = 0
pred = [0]*len(test)
sum = np.sum(weights)
weights = weights/sum

index = 0
for model in models:
  predict  = model.predict(testData)
  for i  in range (len(test)):
    pred[i] += weights[index] * predict[i][0]
  index+=1

In [12]:
print("Ensemble test result: mean_absolute_error=", mae(test['rating'], pred), "mean_square_error=", mse(test['rating'], pred), "rmse=", rmse(test['rating'], pred))

Ensemble test result: mean_absolute_error= 0.7369612999218934 mean_square_error= 0.879383062500814 rmse= tf.Tensor(0.937754265519925, shape=(), dtype=float64)
